In [1]:
import numpy as np
import pandas as pd
import json

In [21]:
df = pd.read_json('0331IG.json') # saves as dataframe

In [24]:
df.head()

,image,objects
0,"{'fileName': '20220331_SD02_3464.JPG', 'detail...","[{'bndbox': {'minX': 0.29847, 'minY': 0.496257..."
1,"{'fileName': '20220331_SD02_3472.JPG', 'detail...","[{'bndbox': {'minX': 0.45220899999999997, 'min..."
2,"{'fileName': '20220331_SD02_3499.JPG', 'detail...","[{'bndbox': {'minX': 0.31612199999999996, 'min..."
3,"{'fileName': '20220331_SD02_3510.JPG', 'detail...","[{'bndbox': {'minX': 0.258353, 'minY': 0.30374..."
4,"{'fileName': '20220331_SD02_3359.JPG', 'detail...","[{'bndbox': {'minX': 0.248725, 'minY': 0.15508..."


In [39]:
df['image'][0]

{'fileName': '20220331_SD02_3464.JPG',
 'details': {'folderName': '0331IG',
  'width': 3648.0,
  'height': 5472.0,
  'depth': 3}}

In [40]:
img = df['image'].to_dict()

# Extract file names
names = []
for i in range(0,len(img)):
    names.append(img[i]['fileName'])
    
# Extract orientation
orientations = []
for i in range(0,len(img)):
    if img[i]['details']['width'] < img[i]['details']['height']:
        orientation = 'portrait'
        orientations.append(orientation)
    else: 
        orientation = 'landscape'
        orientations.append(orientation)

In [108]:
jpg_df = pd.DataFrame({'file_name': names, 'orientation': orientations})
jpg_df.head()

,file_name,orientation
0,20220331_SD02_3464.JPG,portrait
1,20220331_SD02_3472.JPG,landscape
2,20220331_SD02_3499.JPG,portrait
3,20220331_SD02_3510.JPG,portrait
4,20220331_SD02_3359.JPG,portrait


In [75]:
obj = df['objects'].to_dict()
obj
obj[0][0]['bndbox']
obj[0][0]['category']['name']

'42K Face'

In [110]:
coord_list = []
bndbox_name_list = []
for i in range(len(obj)):
    coord = []
    bndbox_name = []
    for j in range(len(obj[i])):
        category_name = obj[i][j]['category']['name']
        bndbox_dict = obj[i][j]['bndbox']
        bndbox_tuple = (bndbox_dict['minX'], bndbox_dict['minY'], bndbox_dict['maxX'], bndbox_dict['maxY'])
        
        coord.append(bndbox_tuple)
        bndbox_name.append(category_name)
    coord_list.append(coord)
    bndbox_name_list.append(bndbox_name)

bndbox_df = pd.DataFrame({'bndbox_name': bndbox_name_list, 'coordinates': coord_list})
bndbox_df.head()


,bndbox_name,coordinates
0,"[42K Face, 42K Body, 68E Face, 68E Body, Unkno...","[(0.29847, 0.496257, 0.5536139999999999, 0.672..."
1,"[Unknown Face, Unknown Body]","[(0.45220899999999997, 0.026510000000000002, 0..."
2,"[167I Face, 167I Body]","[(0.31612199999999996, 0.36363599999999996, 0...."
3,"[Unknown Face, Unknown Body]","[(0.258353, 0.303743, 0.497451, 0.479143999999..."
4,"[167I Face, 167I Body]","[(0.248725, 0.15508000000000002, 0.654709, 0.3..."


In [131]:
merged_df = pd.concat([jpg_df['file_name'], bndbox_df], axis = 1)
merged_df = merged_df.explode(['bndbox_name', 'coordinates'])
merged_df.head(15)

,file_name,bndbox_name,coordinates
0,20220331_SD02_3464.JPG,42K Face,"(0.29847, 0.496257, 0.5536139999999999, 0.672727)"
0,20220331_SD02_3464.JPG,42K Body,"(0.25193499999999996, 0.48984000000000005, 0.6..."
0,20220331_SD02_3464.JPG,68E Face,"(0.466962, 0.22246000000000002, 0.978854, 0.55..."
0,20220331_SD02_3464.JPG,68E Body,"(0.25995799999999997, 0.22139000000000003, 1.0..."
0,20220331_SD02_3464.JPG,Unknown Face,"(0.0, 0.12834199999999998, 0.33858699999999997..."
0,20220331_SD02_3464.JPG,Unknown Body,"(0.0, 0.12834199999999998, 0.333773, 1.0)"
1,20220331_SD02_3472.JPG,Unknown Face,"(0.45220899999999997, 0.026510000000000002, 0...."
1,20220331_SD02_3472.JPG,Unknown Body,"(0.44498000000000004, 0.025304999999999998, 0...."
2,20220331_SD02_3499.JPG,167I Face,"(0.31612199999999996, 0.36363599999999996, 0.6..."
2,20220331_SD02_3499.JPG,167I Body,"(0.003209, 0.36149699999999996, 0.773455, 0.90..."


In [ ]:

for entry in list_of_lists:
    bndbox_name = []
    coord = []
    for k, v in entry:
        bndbox_name.append(k)
        coord.append(v)
    bndbox_name

In [100]:
with open('0331IG.json') as f:
    data = json.load(f)
f.close()

print(type(data))
print(data)

<class 'list'>
[{'image': {'fileName': '20220331_SD02_3464.JPG', 'details': {'folderName': '0331IG', 'width': 3648.0, 'height': 5472.0, 'depth': 3}}, 'objects': [{'bndbox': {'minX': 0.29847, 'minY': 0.496257, 'maxX': 0.553614, 'maxY': 0.672727}, 'category': {'name': '42K Face', 'color': '#6C167F'}, 'tags': [], 'parts': []}, {'bndbox': {'minX': 0.251935, 'minY': 0.48984, 'maxX': 0.609778, 'maxY': 0.869519}, 'category': {'name': '42K Body', 'color': '#2BC9F1'}, 'tags': [], 'parts': []}, {'bndbox': {'minX': 0.466962, 'minY': 0.22246, 'maxX': 0.978854, 'maxY': 0.552941}, 'category': {'name': '68E Face', 'color': '#3A8957'}, 'tags': [], 'parts': []}, {'bndbox': {'minX': 0.259958, 'minY': 0.22139, 'maxX': 1.0, 'maxY': 0.954011}, 'category': {'name': '68E Body', 'color': '#DCA9BE'}, 'tags': [], 'parts': []}, {'bndbox': {'minX': 0.0, 'minY': 0.128342, 'maxX': 0.338587, 'maxY': 0.409626}, 'category': {'name': 'Unknown Face', 'color': '#7A47F8'}, 'tags': [], 'parts': []}, {'bndbox': {'minX': 0.0

In [104]:
data[0][['image']]

TypeError: unhashable type: 'list'

In [16]:
data['image']['details']['width']

5472.0

In [8]:
obj_data= data['objects'][0]

In [9]:
obj_data['category']

{'name': 'monkey1', 'color': '#8517DF'}

In [10]:
obj_data['parts']

[{'polygon': [0.381772,
   0.155856,
   0.383684,
   0.14247,
   0.397068,
   0.143426,
   0.404079,
   0.155856,
   0.400255,
   0.163505,
   0.391969,
   0.164462],
  'category': {'name': 'left eye', 'color': '#5EC256'},
  'tags': [],
  'parts': []},
 {'bndbox': {'minX': 0.362014,
   'minY': 0.166374,
   'maxX': 0.384959,
   'maxY': 0.217051},
  'category': {'name': 'nose', 'color': '#9AEDDC'},
  'tags': [],
  'parts': []}]

In [11]:
df2 = pd.DataFrame(data)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
df2["image"]

In [ ]:
df2["objects"]

In [ ]:
for things in df2["image"]:
    print(things['details'])
    print(things['fileName'])

In [ ]:
for things in df2["objects"]:
    print(things)

In [12]:
df_try= pd.read_json("annotations.json")

In [13]:
type(df_try.objects)

pandas.core.series.Series

In [14]:
bn = pd.DataFrame(df_try.objects.values.tolist())['']
pd.DataFrame.from_records(bn)

KeyError: ''

In [ ]:
df_try.objects.values.tolist()